# Twitter Query Expansion
In this demo file, the project **Twitter Query Expansion** is ...

## Download Models
First, different models for text processing and word embeddings are downloaded. 

For text processing SpaCy is utilized. The word embeddings of Word2Vec and Fasttext are 

In [1]:
SPACY_MODEL = "de_core_news_lg"

In [ ]:
import subprocess

# download the respective SpaCy model
subprocess.run(f"python -m spacy download {SPACY_MODEL}", shell=True)

Download FastText model

In [ ]:
# Download german model from fasttext website
subprocess.run("wget -P ./data/fasttext https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.de.300.bin.gz", shell=True)

# unzip the fasttext model
subprocess.run(f"gunzip -d ./models/fasttext/cc.de.300.bin.gz", shell=True)

Download Word2Vec model

In [ ]:
# Download german model from devmount website
subprocess.run("wget -P ./models/fasttext https://cloud.devmount.de/d2bc5672c523b086/german.model", shell=True)

## Define Queries
The queries on which to evaluate the query expansion are stated.

In [1]:
QUERIES = [
    "Ist die große Koalition gescheitert unter Merkel? #Groko #SPD #CDU",
    "Lauterbach Deutschland Corona-Maßnahmen #Impfung",
    "@bundestag Bundestagswahl 2021 Ergebnisse",
    "Europäische Union #Brexit Boris Johnson",
    "Gesetzliche Rentenversicherung Reform #Rentenreform #RenteMit67",
    "Klimapolitik Deutschland #Grüne",
    "Asylpolitik Merkel",
    "Deutsch-französische Beziehungen schwierig",
    "Soll man Soli abschaffen",
    "Bundeswehr Afghanistan Krieg stoppen",
    "Deutschland Energiewende unter SPD und CDU"
]

SPACY_MODEL = "de_core_news_lg"

## Set Pipeline Parameters
Modify the following parameters to use different options for the word embeddings and elastic search query.

`pos_list:` \
`['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'CONJ', 'DET', 'EOL', 'IDS', 'INTJ', 'NAMES', 'NOUN', 'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SPACE', 'SYM', 'VERB', 'X']`

`entity_list: ` \
`['LOC', 'MISC', 'ORG', 'PER']`

In [2]:
EMBEDDING_PARAMS = {
    "type": "fasttext",
    "pos_list": ["NOUN","ADJ","VERB", "PROPN"],
    "entity_list": ['LOC', 'ORG'],
    "hashtag": True,
    "user": False,
    "num_nearest_terms": 5
}

ELASTIC_PARAMS = {
    "index": "tweets",
    "num_of_tweets": 10,
    "retweet": False,
    "hashtag_boost": 0.5,
    "tweet_range": ("2021-01-01", "2023-01-01")
}

## Execute Pipeline
Run the Pipeline - the results are stored in the `/out` directory.

In [3]:
# run pipeline
from scripts import pipeline

res = pipeline.run(
    queries=QUERIES, 
    spacy_model=SPACY_MODEL,
    embedding_params=EMBEDDING_PARAMS,
    elastic_params=ELASTIC_PARAMS)

Processing text using SpaCy...
Loading fasttext model...
Connecting to Elastic Search...
Successfully connected to https://localhost:9200
Retrieving Tweets...
Writing results to out/fasttext/11-01-23_10-37-42
Done!


## Inspect Results
Load results and have a look through the retrieved Tweets. 

In [5]:
for tweets, i in zip(res,range(len(QUERIES))):
    print("Query:", QUERIES[i], "\n")
    for tweet in tweets["tweets"]:
        print("->", tweet["_source"]["txt"])

Query: Ist die große Koalition gescheitert unter Merkel? #Groko 

-> Im Juni hat die #GroKo  #union #spd mit #afd den Antrag der Opposition abgelehnt Ortskräfte in #Afghanistan ein Visa zu erteilen 😤

Genau das ist Standard! Anträge Opposition werden immer und prinzipiell abgelehnt 😤😤
#fraktionszwang #parteitaktik #undemokratisch
-> #GROKO #SPD #CDU mauert! Aufklärung zum #Lehrergate - #Transparenz und Offenheit sucht man vergebens!  Vertuschungs-Mails von ganz oben sollten Ministerin @BettinaMartin4 `s Versagen kaschieren.
👉Nicht mit UNS @AfDFraktion_MV 

#nurnochAfD #MV #ltw21
https://t.co/n5uE7c1hQW
-> Hier unser Antrag zur Aufnahme von Ortskräften aus #Afghanistan - von CDU/CSU und SPD sowie AfD haben im Juni abgelehnt, Enthaltung der FDP. Wer übernimmt die Verantwortung in der #GroKo ??? Der Antrag wurde im April !!! formuliert und eingebracht- im April @GrueneBundestag https://t.co/Tat5QP8yJF
-> #GroKo mit Reserverad und dem „mit Abstand schlechtesten Koalitionsvertrag“ (@Lischka